In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bokeh
import glob
import os
import warnings
import numpy as np
from bokeh.models import ColumnDataSource, Band, HoverTool
from bokeh.plotting import figure, show
from tqdm.notebook import tqdm
from utils.measuring_performance import (
    get_prediction, plot_histogram_by_class, plot_loss_per_epoch, plot_pr_curve, plot_roc_curve)
from utils.misc import build_files_list, dump_pickle, load_pickle
from utils.sound_utils import extract_signal_features, generate_dataset, load_sound_file
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
bokeh.io.output_notebook()
warnings.filterwarnings('ignore')
np.random.seed(42)

Loading BokehJS ...

In [3]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib
tf.set_random_seed(42)

In [4]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

The boolean variable `COMBINE_MACHINE_ID` allows you to decide whether to perform modeling separately for each machine ID, or combine all data from the same machine type to perform modeling.

In [5]:
DATA_PATH = '../../data/mimii-anomaly-detection'
IMAGE_PATH = './img'
MODEL_PATH = './models'
MERGE_MACHINE_ID = True

os.makedirs(os.path.join(DATA_PATH, 'dataset'), exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

file_paths = sorted(glob.glob(DATA_PATH + '/*/*' if MERGE_MACHINE_ID else DATA_PATH + '/*/*/*'))

# Data Splitting and Preprocessing

In [6]:
file_path = file_paths[0] # the first machine ID or machine type selected 
file_path_split = file_path.split('/')
suffix = '_'.join(['', file_path_split[-1], file_path_split[-2]])

if MERGE_MACHINE_ID:
    print('db: {}, machine type: {}'.format(file_path_split[-2], file_path_split[-1]))

else:
    print('db: {}, machine type: {}, machine id: {}'.format(file_path_split[-3], file_path_split[-2], file_path_split[-1]))
    suffix = '_'.join([suffix, file_path_split[-3]]) 

db: 6dB, machine type: fan


Because unsupervised learning is used, all anomalous labels are assigned to the test set.

In [7]:
normal_files, abnormal_files = build_files_list(root_dir=file_path)
normal_labels = np.zeros(len(normal_files))
abnormal_labels = np.ones(len(abnormal_files))

train_files, test_files, train_labels, test_labels = train_test_split(normal_files, normal_labels, train_size=0.8, random_state=42, shuffle=True)

test_files = np.concatenate((test_files, abnormal_files), axis=0)
test_labels = np.concatenate((test_labels, abnormal_labels), axis=0)

test_indices = np.arange(len(test_files))
np.random.shuffle(test_indices)

test_files = test_files[test_indices]
test_labels = test_labels[test_indices]

print('Train set has {} signals including abnormal {:.0f} signals, but test set has {} signals including abnormal {:.0f} signals.'.format(
    train_labels.shape[0], train_labels.sum(), test_labels.shape[0], test_labels.sum()))

Train set has 3260 signals including abnormal 0 signals, but test set has 2290 signals including abnormal 1475 signals.


In [8]:
dataset = {
    'train_files': train_files,
    'test_files': test_files,
    'train_labels': train_labels,
    'test_labels': test_labels
}

for key, values in dataset.items():
    file_name = os.path.join(DATA_PATH, 'dataset', key + suffix + '.txt')
    with open(file_name, 'w') as f:
        for item in values:
            f.write(str(item) + '\n')

In [9]:
n_fft = 2048
hop_length = 512
n_mels = 64
frames = 5

train_data_path = os.path.join(DATA_PATH, 'dataset', 'train_data' + suffix + '.pkl')

if os.path.exists(train_data_path):
    print('Train data already exists, loading from file...')
    train_data = load_pickle(train_data_path)
        
else:
    train_data = generate_dataset(train_files, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames)
    print('Saving train data to disk...')
    dump_pickle(train_data_path, train_data)
    print('Done.')

print(f'Train data has a {train_data.shape} shape.')

Train data already exists, loading from file...
Train data has a (1007340, 320) shape.


# Model Training and Prediction with AutoEncoder

In [10]:
def autoencoder(input_dims, model_name=None):
    input_layer = Input(shape=(input_dims,))
    h = Dense(64, activation='relu')(input_layer)
    h = Dense(64, activation='relu')(h)
    h = Dense(8, activation='relu')(h)
    h = Dense(64, activation='relu')(h)
    h = Dense(64, activation='relu')(h)
    h = Dense(input_dims, activation=None)(h)
    
    return Model(inputs=input_layer, outputs=h, name=model_name)

In [11]:
model_name = 'AutoEncoder'
model = autoencoder(n_mels * frames, model_name=model_name)
print(model.summary())

gpu_count = len(get_available_gpus())
if gpu_count > 1:
    model = multi_gpu_model(model, gpus=gpu_count)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                20544     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
___

In [ ]:
%%time
batch_size = 512
epochs = 200

model.compile(
    optimizer=Adam(learning_rate=1e-03),
    loss='mean_squared_error'
)

history = model.fit(
    train_data, 
    train_data,
    batch_size=batch_size,
    epochs=epochs,
    verbose=False,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],
    validation_split=0.1,
    shuffle=True
)

model.save(os.path.join(MODEL_PATH, model_name +  suffix + '.h5'))

In [ ]:
plot_loss_per_epoch(
    history, model_name=model_name, file_name=os.path.join(IMAGE_PATH, 'ae_model_loss.svg'))

In [ ]:
recon_errors = []

for index in tqdm(range(len(test_files))):
    signal, sr = load_sound_file(test_files[index])
    
    features = extract_signal_features(
        signal, 
        sr, 
        n_fft=n_fft, 
        hop_length=hop_length,
        n_mels=n_mels, 
        frames=frames
    )
    
    predictions = model.predict(features)
    mse = np.mean(np.mean(np.square(features - predictions), axis=1))
    recon_errors.append(mse)

# Model Evaluation

In [ ]:
stack = np.column_stack((range(len(recon_errors)), recon_errors))
score_false = stack[test_labels == 0][:, 1]
score_true = stack[test_labels == 1][:, 1]

plot_histogram_by_class(
    score_false, score_true, bins=[20, 30], model_name=model_name, file_name=os.path.join(IMAGE_PATH, 'ae_recon_error_dist.svg'))

In [ ]:
thr_min = 3.0
thr_max = 8.0

p = figure(plot_width=600, plot_height=400, title='{} - Threshold Range Exploration'.format(model_name), x_axis_label='Samples', y_axis_label='Reconstruction Error')

source = ColumnDataSource(dict(index=stack[test_labels == 0][:, 0], error=stack[test_labels == 0][:, 1]))
p.scatter('index', 'error', fill_alpha=0.6, fill_color='crimson', line_color=None, legend_label='Normal Signals', source=source)

source = ColumnDataSource(dict(index=stack[test_labels == 1][:, 0], error=stack[test_labels == 1][:, 1]))
p.scatter('index', 'error', fill_alpha=0.6, fill_color='indigo', line_color=None, legend_label='Abnormal Signals', source=source)

source = ColumnDataSource(data=dict(index=stack[:, 0], thr_min=np.repeat(thr_min, stack.shape[0]), thr_max=np.repeat(thr_max, stack.shape[0])))

band = Band(base='index', lower='thr_min', upper='thr_max', level='underlay', 
            fill_alpha=0.1, fill_color='magenta', line_color='darkmagenta', line_width=1.0, source=source)
p.add_layout(band)

p.legend.label_text_font_size = '8pt'
p.title.align = 'center'
p.title.text_font_size = '12pt'

p.add_tools(HoverTool(tooltips=[('index', '@index'), ('error', '@error')]))
show(p)

p.output_backend = 'svg'
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, 'ae_thr_range_expl.svg'))

In [ ]:
threshold = 3.0
preds = get_prediction(stack[:, 1], threshold=threshold)

print('<{0}> Accuracy: {1:.2%}, Precision: {2:.2%}, Recall: {3:.2%}, F1: {4:.2%}'.format(
    model_name, accuracy_score(test_labels, preds), precision_score(test_labels, preds), 
    recall_score(test_labels, preds), f1_score(test_labels, preds)))

In [ ]:
plot_roc_curve(
    roc_curve(test_labels, recon_errors), roc_auc_score(test_labels, recon_errors), 
    model_name=model_name, file_name=os.path.join(IMAGE_PATH, 'ae_roc_curve.svg'))

In [ ]:
plot_pr_curve(
    precision_recall_curve(test_labels, recon_errors), average_precision_score(test_labels, recon_errors), 
    model_name=model_name, file_name=os.path.join(IMAGE_PATH, 'ae_pr_curve.svg'))

In [ ]:
def generate_error_types(df, ground_truth_col='Ground Truth', prediction_col='Prediction', normal_label=0.0, anomaly_label=1.0):
    """
    Compute false positive and false negatives columns based on the prediction
    and ground truth columns from a dataframe.
    
    PARAMS
    ======
        df (dataframe)
            Dataframe where the ground truth and prediction columns are available
        ground_truth_col (string)
            Column name for the ground truth values. Defaults to "Ground Truth"
        prediction_col (string)
            Column name for the predictied values. Defaults to "Prediction"
        normal_label (object)
            Value taken by a normal value. Defaults to 0.0
        anomaly_label (object)
            Value taken by an abnormal value. Defaults to 1.0
            
    RETURNS
    =======
        df (dataframe)
            An updated dataframe with 4 new binary columns for TP, TN, FP and FN.
    """
    df['TP'] = 0
    df['TN'] = 0
    df['FP'] = 0
    df['FN'] = 0
    df.loc[(df[ground_truth_col] == df[prediction_col]) & (df[ground_truth_col] == normal_label), 'TP'] = 1
    df.loc[(df[ground_truth_col] == df[prediction_col]) & (df[ground_truth_col] == anomaly_label), 'TN'] = 1
    df.loc[(df[ground_truth_col] != df[prediction_col]) & (df[ground_truth_col] == normal_label), 'FP'] = 1
    df.loc[(df[ground_truth_col] != df[prediction_col]) & (df[ground_truth_col] == anomaly_label), 'FN'] = 1
    
    return df

def plot_curves(FP, FN, nb_samples, threshold_min, threshold_max, threshold_step):
    """
    Plot the false positives and false negative samples number depending on a given threshold.
    
    PARAMS
    ======
        FP (dataframe)
            Number of false positives depending on the threshold
        FN (dataframe)
            Number of false negatives depending on the threshold
        threshold_min (float)
            Minimum threshold to plot for
        threshold_max (float)
            Maximum threshold to plot for
        threshold_step (float)
            Threshold step to plot these curves
    """
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)
    
    min_FN = np.argmin(FN)
    min_FP = np.where(FP == np.min(FP))[0][-1]
    plot_top = max(FP + FN) + 1

    # Grid customization:
    major_ticks = np.arange(threshold_min, threshold_max, 1.0 * threshold_step)
    minor_ticks = np.arange(threshold_min, threshold_max, 0.2 * threshold_step)
    ax.set_xticks(major_ticks);
    ax.set_xticks(minor_ticks, minor=True);
    ax.grid(which='minor', alpha=0.5)
    ax.grid(which='major', alpha=1.0, linewidth=1.0)
    
    # Plot false positives and false negatives curves
    plt.plot(np.arange(threshold_min, threshold_max + threshold_step, threshold_step), FP, label='False positive', color='tab:red')
    plt.plot(np.arange(threshold_min, threshold_max + threshold_step, threshold_step), FN, label='False negative', color='tab:green')

    # Finalize the plot with labels and legend:
    plt.xlabel('Reconstruction error threshold (%)', fontsize=16)
    plt.ylabel('# Samples', fontsize=16)
    plt.legend()
    
    
def print_confusion_matrix(confusion_matrix, class_names, figsize = (4,3), fontsize=14):
    """
    Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix,
    as a heatmap.
    
    PARAMS
    ======
        confusion_matrix (numpy.ndarray)
            The numpy.ndarray object returned from a call to 
            sklearn.metrics.confusion_matrix. Similarly constructed 
            ndarrays can also be used.
        class_names (list)
            An ordered list of class names, in the order they index the given
            confusion matrix.
        figsize (tuple)
            A 2-long tuple, the first value determining the horizontal size of
            the ouputted figure, the second determining the vertical size.
            Defaults to (10,7).
        fontsize: (int)
            Font size for axes labels. Defaults to 14.
        
    RETURNS
    =======
        matplotlib.figure.Figure: The resulting confusion matrix figure
    """
    # Build a dataframe from the confusion matrix passed as argument:
    df_cm = pd.DataFrame(confusion_matrix, 
                         index=class_names, 
                         columns=class_names)
    
    # Plot the confusion matrix:
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", annot_kws={"size": 16}, cmap='viridis')
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    # Figure customization:
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label', fontsize=16)
    plt.xlabel('Predicted label', fontsize=16)
    
    return fig

In [ ]:
import pandas as pd
thresholds = np.arange(thr_min, thr_max + threshold_step, threshold_step)

df = pd.DataFrame(columns=['Signal', 'Ground Truth', 'Prediction', 'Reconstruction Error'])
df['Signal'] = test_files
df['Ground Truth'] = test_labels
df['Reconstruction Error'] = recon_errors

FN = []
FP = []
for th in thresholds:
    df.loc[df['Reconstruction Error'] <= th, 'Prediction'] = 0.0
    df.loc[df['Reconstruction Error'] > th, 'Prediction'] = 1.0
    df = generate_error_types(df)
    FN.append(df['FN'].sum())
    FP.append(df['FP'].sum())
        
plot_curves(FP, FN, nb_samples=df.shape[0], threshold_min=thr_min, threshold_max=thr_max, threshold_step=threshold_step)

In [ ]:
import seaborn as sns

th = 6.275
df.loc[df['Reconstruction Error'] <= th, 'Prediction'] = 0.0
df.loc[df['Reconstruction Error'] > th, 'Prediction'] = 1.0
df['Prediction'] = df['Prediction'].astype(np.float32)
df = generate_error_types(df)
tp = df['TP'].sum()
tn = df['TN'].sum()
fn = df['FN'].sum()
fp = df['FP'].sum()

from sklearn.metrics import confusion_matrix
df['Ground Truth'] = 1 - df['Ground Truth']
df['Prediction'] = 1 - df['Prediction']
print_confusion_matrix(confusion_matrix(df['Ground Truth'], df['Prediction']), class_names=['abnormal', 'normal']);

